## load libs

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from process_images import *
from pystripe.core import *
import matplotlib.pyplot as plt
def plot_images(img_list: List[ndarray], img_labels: List[str], vmax: int):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    for idx, (im, label) in enumerate(zip(img_list, img_labels)):
        axes[idx].imshow(im, cmap='gray', vmin=0, vmax=vmax)
        axes[idx].set_title(label)
    plt.tight_layout()
    plt.show()

In [13]:
from parallel_image_processor import *
tsv_volume = TSVVolume.load(r'E:\20230510_13_34_13_SM230308_05_LS_15x_800z_MIP_stitched\Ex_488_Em_525_MIP_xml_import_step_5.xml')
shape: Tuple[int, int, int] = tsv_volume.volume.shape  # shape is in z y x format
img = tsv_volume.imread(
    VExtent(
        tsv_volume.volume.x0, tsv_volume.volume.x1,
        tsv_volume.volume.y0, tsv_volume.volume.y1,
        tsv_volume.volume.z0 + shape[0]//2, tsv_volume.volume.z0 + shape[0]//2 + 1),
    tsv_volume.dtype)[0]
parallel_image_processor(
    source=TSVVolume.load(r'/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_xml_import_step_5.xml'),
    destination=r"/data/20230419_17_34_03_SM221011_06_LS_15x_800z_stitched/Ex_488_Em_525_tif",
    fun=process_img,
    kwargs={'bleach_correction_frequency': 0.0005, 'bleach_correction_max_method': False, 'bleach_correction_y_slice_max': None, 'threshold': None, 'sigma': (4000.0, 4000.0), 'bidirectional': True, 'lightsheet': False, 'percentile': 0.25, 'rotate': 90, 'convert_to_8bit': False, 'bit_shift_to_right': 8, 'tile_size': (39220, 28056), 'd_type': 'uint16', "verbose": True},
    source_voxel=(0.8, 0.4, 0.4),
    target_voxel=20,
    max_processors=1
)

326.0000698052347


In [ ]:
merge_all_channels(
    [
        Path(r"/data/20230411_12_44_26_SW220414_01_LS_15x_800z_stitched/Ex_488_Em_525_tif"),
        Path(r"/data/20230411_12_44_26_SW220414_01_LS_15x_800z_stitched/Ex_488_Em_525_tif_deconvolved"),
        Path(r"/data/20230411_12_44_26_SW220414_01_LS_15x_800z_stitched/Ex_561_Em_600_tif"),
    ],
    Path(r"/data/20230411_12_44_26_SW220414_01_LS_15x_800z_stitched/merged"),
    compression=None,
    right_bit_shift=(0, 8, 5)
)
